(Enning, 04.01.2023)
## Internet-Protokol IPv6

- Jeder Teilnehmer (Node) hat mindestens (meistens mehrere) eine IPv6-Adresse
- Jede Adresse hat 16 Bytes!

In [1]:
f' mit 16 Bytes kann man {2**(16*8):,} Nodes eindeutige Adressen geben'

' mit 16 Bytes kann man 340,282,366,920,938,463,463,374,607,431,768,211,456 Nodes eindeutige Adressen geben'

- IPv6 Adressen werden in der Form `nnnn:nnnn:nnnn:nnnn:nnnn:nnnn:nnnn:nnnn` geschrieben, wobei `nnnn` jeweils die Hexadezimal-Dezimaldarstellung eines Word (2 Bytes) ist

IPv6 Adressen meines privaten PC

    manfred@WAG-0C-37803997014:~$ ip -6 a
    1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 state UNKNOWN qlen 1000
        inet6 ::1/128 scope host 
           valid_lft forever preferred_lft forever
    2: enp3s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 state UP qlen 1000
        inet6 2a02:908:1020:7160:8cad:7bc8:50e5:20e8/128 scope global dynamic noprefixroute 
           valid_lft 7197sec preferred_lft 3597sec
        inet6 2a02:908:1020:7160:d974:ed61:ff05:2a4e/64 scope global temporary dynamic 
           valid_lft 7198sec preferred_lft 3598sec
        inet6 2a02:908:1020:7160:7c49:7248:d2d5:1bf3/64 scope global dynamic mngtmpaddr noprefixroute 
           valid_lft 7198sec preferred_lft 3598sec
        inet6 fe80::8cad:7bc8:50e5:20e8/64 scope link noprefixroute 
           valid_lft forever preferred_lft forever
    7: br-d4c9dd51a90a: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 state UP 
        inet6 fe80::42:3bff:fe3c:bba7/64 scope link 
           valid_lft forever preferred_lft forever
    9: vpn0: <NO-CARRIER,POINTOPOINT,MULTICAST,NOARP,UP> mtu 1300 state DOWN qlen 500
        inet6 fe80::6542:ca7b:61d8:6cad/64 scope link stable-privacy 
           valid_lft forever preferred_lft forever
    13: vethe9ed70e@if12: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 state UP 
        inet6 fe80::30d1:7ff:fe7a:a31b/64 scope link 
           valid_lft forever preferred_lft forever


### Domain Name System (DNS)

- Auch IPv6 Adressen können mit DNS nachgeschlagen werden (AAAA)


In [3]:
# Wir importieren einen DNS-Resolver
import dns.resolver

In [4]:
# und nutzen diesen, um die IPv4 (A-Record) Adresse zu erfragen
answer = dns.resolver.resolve('google.com', 'AAAA')
list(answer)

[<DNS IN AAAA rdata: 2a00:1450:4001:831::200e>]

- Die IPv6-Adresse zur Domain google.com ist `2a00:1450:4001:82f::200e`
- Führende Nullen können weggelassen werden; `::` entspricht  `:0:0:0:` (Auffüllen)
- Die Adresse gehört zu einem Adressraum, `2a00:1450:4001` der bei der [IANA](https://en.wikipedia.org/wiki/Internet_Assigned_Numbers_Authority) registriert ist
- Mit IPv6 kann "jedem Sandkorn" auf der Welt eine eigene IP-Adresse gegeben werden
- IPv6 ist die Grundlage des "Internet der Dinge", z.B. [`6LoWPAN`](https://de.wikipedia.org/wiki/6LoWPAN)
- Tricksereien mit Subnetzen sind nicht mehr notwendig


#### IPv6 Routing Tabelle

    manfred@WAG-0C-37803997012:~ $ ip -6 r
    ::1 dev lo proto kernel metric 256 pref medium
    fd01::/64 dev br-62b4507a941a proto kernel metric 256 pref medium
    fd78:3214:6290:d772::/64 dev eth1 proto kernel metric 100 pref medium
    fe80::/64 dev br-62b4507a941a proto kernel metric 256 pref medium
    fe80::/64 dev veth6a099cb proto kernel metric 256 pref medium
    fe80::/64 dev veth0bd49ee proto kernel metric 256 pref medium
    fe80::/64 dev veth437bdec proto kernel metric 256 pref medium
    fe80::/64 dev veth9e13ca7 proto kernel metric 256 pref medium
    fe80::/64 dev eth1 proto kernel metric 1024 pref medium
    manfred@WAG-0C-37803997012:~ $ 

    fd01::/64 bedeutet: 
        Alle Adressen, die mit fd01:0000:0000:0000 beginnen (64 bits) gehen auf das Interface br-...


### DHCPv6
- Bei IPv6 gibt es keine Adress-"Not" mehr
- Trotzdem gibt es den Wunsch nach privaten Netzen, in denen Außenschnittstellen kontrolliert werden
- Deshalb gibt es bei IPv6 Adressen mit globalen Scope aber auch Adressen mit begrenzter Routingerlaubnis
- Solche Adressen werden entweder durch einen DHCPv6 Adressen ausgegegeben oder auf dem Client selbst gebaut
- DCHPv6 funktioniert analog zu DCHP

### Router Advertisement (RA), [Neighbor Discovery Protocol](https://de.wikipedia.org/wiki/Neighbor_Discovery_Protocol)

- eine kontrollierte Vergabe von IPv6 Adressen ist nicht nötig
- Es reicht, dem Client den "Prefix" zu nennen, in dem er Adressen erzeugen soll (Router Advertisement)
- Durch Anhängen einer zufälligen Zahlenfolge (anonym) oder einer Zahlenfolge, die aus der MAC-Adresse generiert wird (nachverfolgbar!) entsteht eine eindeutige IPv6 Adresse